In [ ]:
import networkx as nx
import pickle
from utils import *
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import os

In [3]:
def find_files(folder, substring):
    return [f for f in os.listdir(folder) if substring.lower() in f.lower()]

df = df2 = None

search_sub = 'comment'
matching_files = find_files("/sciclone/home/twford/repos/reddit/data/raw_csv/", search_sub)
print(matching_files)
for f in matching_files:
    if df is None:
        print("trying to read")
        df = pd.read_csv("/sciclone/home/twford/repos/reddit/data/raw_csv/"+f)
        print("read one")
    else:
        df = pd.concat([df, pd.read_csv("/sciclone/home/twford/repos/reddit/data/raw_csv/"+f)])
df["type"] = 'comment'

In [4]:
search_sub = 'submission'
matching_files = find_files("/sciclone/home/twford/repos/reddit/data/raw_csv/", search_sub)
for f in matching_files:
    if df2 is None:
        df2 = pd.read_csv("/sciclone/home/twford/repos/reddit/data/raw_csv/"+f)
    else:
        df2 = pd.concat([df, pd.read_csv("/sciclone/home/twford/repos/reddit/data/raw_csv/"+f)])
df2["type"] = search_sub

df = pd.concat([df,df2])

[('RS_2021-01.zstscraped.json', 'RC_2021-01.zstscraped.json'),
 ('RS_2021-02.zstscraped.json', 'RC_2021-02.zstscraped.json'),
 ('RS_2021-03.zstscraped.json', 'RC_2021-03.zstscraped.json'),
 ('RS_2021-04.zstscraped.json', 'RC_2021-04.zstscraped.json'),
 ('RS_2021-05.zstscraped.json', 'RC_2021-05.zstscraped.json'),
 ('RS_2021-06.zstscraped.json', 'RC_2021-06.zstscraped.json'),
 ('RS_2021-07.zstscraped.json', 'RC_2021-07.zstscraped.json'),
 ('RS_2021-08.zstscraped.json', 'RC_2021-08.zstscraped.json'),
 ('RS_2021-09.zstscraped.json', 'RC_2021-09.zstscraped.json'),
 ('RS_2021-10.zstscraped.json', 'RC_2021-10.zstscraped.json'),
 ('RS_2021-11.zstscraped.json', 'RC_2021-11.zstscraped.json'),
 ('RS_2021-12.zstscraped.json', 'RC_2021-12.zstscraped.json'),
 ('RS_2022-01.zstscraped.json', 'RC_2022-01.zstscraped.json'),
 ('RS_2022-02.zstscraped.json', 'RC_2022-02.zstscraped.json'),
 ('RS_2022-03.zstscraped.json', 'RC_2022-03.zstscraped.json'),
 ('RS_2022-04.zstscraped.json', 'RC_2022-04.zstscraped.

In [5]:
data = df

  0%|          | 0/22 [00:00<?, ?it/s]

Computing Centrality Measures:   0%|          | 0/7 [00:00<?, ?it/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f95902f5750>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f95902f5750>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f95902f5750>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean

KeyboardInterrupt: 

In [ ]:
data['created_utc'] = pd.to_datetime(data['created_utc'], errors="coerce", utc=True, unit='s')

data = data.dropna(subset=['created_utc'])

# Extract year-month for grouping
data['year_month'] = data['created_utc'].dt.to_period('M')

start_date = pd.Timestamp("2011-08-01")
end_date = pd.Timestamp("2024-12-31")
date_range = pd.date_range(start=start_date, end=end_date, freq="MS")  # MS = Month Start

In [ ]:
# Process one month at a time
G = nx.DiGraph()
local_running_data = []
global_data = []

for month, month_df in tqdm(data.groupby('year_month')):
    # print(f"Processing {month}...")

    # Create directed graph
    L = nx.DiGraph()

    # Add edges based on user interactions via comments
    for _, row in month_df.iterrows():
        parent = row['parent_id']
        commenter = row['author']

        if pd.notna(parent) and pd.notna(commenter):
            replying_to = data[(data['comment_id'] == row['parent_id']) & (data['depth'] == 0)]['author']
            if len(replying_to) > 0:
                G.add_edge(replying_to.values[0], commenter)
                L.add_edge(replying_to.values[0], commenter)

    dat = compute_graph_measures(L)
    local_running_data.append(dat)

    dat = compute_graph_measures(G)
    global_data.append(dat)

    # Save the graph
    graph_filename = f"graphs/local_reddit_network_{month}.graphml"
    nx.write_graphml(L, graph_filename)
    # print(f"Saved {graph_filename}")

    # Save the graph
    graph_filename = f"graphs/global_reddit_network_{month}.graphml"
    nx.write_graphml(G, graph_filename)
    # print(f"Saved {graph_filename}")

    # Save to JSON
    json_filename = "local_graph_measures_extended.json"
    with open(json_filename, "w") as f:
        json.dump(local_running_data, f, indent=4)

    # Save to JSON
    json_filename = "global_graph_measures_extended.json"
    with open(json_filename, "w") as f:
        json.dump(global_data, f, indent=4)

print("Processing complete.")

In [ ]:
analysis_df = pd.read_json('local_graph_measures_extended.json')

In [ ]:
start_date = pd.Timestamp("2011-08-01")
end_date = pd.Timestamp("2024-12-31")
date_range = pd.date_range(start=start_date, end=end_date, freq="MS")  # MS = Month Start


In [ ]:
import seaborn as sns

In [ ]:
avg_dc = []
for month in analysis_df.iterrows():
    avg_dc.append(np.mean(list(month[1]['measures']['degree_centrality'].values())))

df = pd.DataFrame({"Date": date_range, "Average Degree Centrality": avg_dc})


# Set publication-style aesthetics
sns.set_theme(style="whitegrid")

# Create the line plot
plt.figure(figsize=(10, 5))
sns.lineplot(data=df, x="Date", y="Average Degree Centrality", color="darkblue", linewidth=2)

# Format x-axis with readable dates
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Year", fontsize=14)
plt.ylabel("Average Degree Centrality", fontsize=14)
plt.title("Local Network Average Degree Centrality (2011-08 to 2024-12)", fontsize=16)

# Remove top and right spines for a clean look
sns.despine()

# Show the plot
plt.show()

In [ ]:
avg_dc = []
for month in analysis_df.iterrows():
    avg_dc.append(np.mean(list(month[1]['measures']['degree_centrality'].values())))

avg_cc = []
for month in analysis_df.iterrows():
    # print(month[1]['measures'].keys())

    avg_cc.append(np.mean(list(month[1]['measures']['betweenness_centrality'].values())))

avg_hub = []
for month in analysis_df.iterrows():
    avg_hub.append(np.mean(list(month[1]['measures']['hubs'].values())))

avg_auth = []
for month in analysis_df.iterrows():
    avg_auth.append(np.mean(list(month[1]['measures']['authorities'].values())))

df = pd.DataFrame({"Date": date_range, "Degree Centrality": avg_dc, "Closeness Centrality":avg_cc, "Hubs": avg_hub, "Authorities": avg_auth})

df = df.melt(id_vars=["Date"], var_name="Centrality Measure", value_name="average")

df["average"] = df["average"] + np.random.uniform(-0.002, 0.002, len(df))

# Set publication-style aesthetics
sns.set_theme(style="whitegrid")

# Create the line plot
plt.figure(figsize=(10, 5))
sns.lineplot(data=df, x="Date", y="average", hue='Centrality Measure', linewidth=2, #style='Centrality Measure',
            alpha=0.5)

# Format x-axis with readable dates
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Year", fontsize=14)
plt.ylabel("Average", fontsize=14)
plt.title("Local Network Centrality Averages (2011-08 to 2024-12)", fontsize=16)
# plt.legend(title="Category", fontsize=12, title_fontsize=14)

# Remove top and right spines for a clean look
sns.despine()

# Show the plot
plt.show()

In [ ]:
avg_dc = []
for month in analysis_df.iterrows():
    avg_dc.append(np.mean(list(month[1]['authorities'].values())))

df = pd.DataFrame({"Date": date_range[:128], "Average Degree Centrality": avg_dc})


# Set publication-style aesthetics
sns.set_theme(style="whitegrid")

# Create the line plot
plt.figure(figsize=(10, 5))
sns.lineplot(data=df, x="Date", y="Average Degree Centrality", color="darkblue", linewidth=2)

# Format x-axis with readable dates
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Date", fontsize=14)
plt.ylabel("Average Authorities", fontsize=14)
plt.title("Community's Average Authorities (2011-08 to 2023-03)", fontsize=16)

# Remove top and right spines for a clean look
sns.despine()

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

# Load the dataset (Assuming a CSV with 'Date' and 'Text' columns)

# Initialize sentiment analyzer
vader = SentimentIntensityAnalyzer()

# Function to compute sentiment score
def get_sentiment(text):
    if pd.isna(text) or not isinstance(text, str):  # Handle missing or non-string values
        return 0
    return vader.polarity_scores(text)["compound"]

In [ ]:
model_path = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment_pipeline = pipeline(model=model_path, tokenizer=model_path, max_length=512, truncation=True)

In [ ]:
def cardiff_get_sentiment(text):
    if pd.isna(text) or not isinstance(text, str):  # Handle missing or non-string values
        return 0

    # Split text into 500-character chunks (ensuring model limit safety)
    chunks = [text[i:i+500] for i in range(0, len(text), 500)]

    sentiments = []
    for chunk in chunks:
        result = sentiment_pipeline(chunk)
        label = result[0]['label']

        if label == "LABEL_2":  # Positive
            sentiments.append(1)
        elif label == "LABEL_1":  # Neutral
            sentiments.append(0)
        elif label == "LABEL_0":  # Negative
            sentiments.append(-1)
    if len(sentiments) != 0:
        sentiment = sum(sentiments) / len(sentiments)
    else:
        sentiment = 0
    return sentiment  # Return average sentiment

In [ ]:
data['selftext'] = data['selftext'].fillna('')
data['body'] = data['body'].fillna('')

In [ ]:
data["content"] = data['selftext'].astype(str) + data['body'].astype(str)

In [ ]:
data["VADER Sentiment"] = data["content"].apply(get_sentiment)

In [ ]:
data["RoBERTa Sentiment"] = data["content"].apply(cardiff_get_sentiment)

In [ ]:
data_subset = data[["year_month","VADER Sentiment", "RoBerta Sentiment"]]

In [ ]:
data_subset = data_subset.melt(id_vars=["year_month"], var_name="Sentiment Measure", value_name="average")


In [ ]:
sentiment_trend = data_subset.groupby(["year_month", "Sentiment Measure"]).mean().reset_index()


In [ ]:
# Set Seaborn theme
sns.set_theme(style="whitegrid")

# Plot sentiment trend over time
plt.figure(figsize=(10, 5))
sns.lineplot(data=sentiment_trend, x="year_month", y="average", hue="Sentiment Measure", color="darkblue", linewidth=2)

# Formatting
plt.xlabel("Year", fontsize=14)
plt.ylabel("Average Sentiment", fontsize=14)
plt.title("Monthly Sentiment Analysis", fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
sns.despine()

# Show the plot
plt.show()

In [ ]:
data["RoBerta Sentiment"] = data["content"].apply(cardiff_get_sentiment)
sentiment_trend = data.groupby(["year_month"])["Sentiment-cardiff"].mean().reset_index()

# Convert back to datetime for plotting
sentiment_trend["year_month"] = sentiment_trend["year_month"].dt.to_timestamp()

# Set Seaborn theme
sns.set_theme(style="whitegrid")

# Plot sentiment trend over time
plt.figure(figsize=(10, 5))
sns.lineplot(data=sentiment_trend, x="year_month", y="Sentiment-cardiff", color="darkblue", linewidth=2)

# Formatting
plt.xlabel("Month", fontsize=14)
plt.ylabel("Average Sentiment", fontsize=14)
plt.title("Monthly Sentiment Analysis", fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
sns.despine()

# Show the plot
plt.show()